# Глава 24. Дополнительные возможности модулей

## Сокрытие данных в модулях

### Минимизация повреждений, причиняемых инструкцией `from *`: `_X` и `__all__`

Как особый случай, существет возможность **начинать имена переменных с одного символа подчеркивания `_`**, чтобы предотвратить их перезаписывание, когда клиент выполняет импортирование модуля инструкцией `from *`.

Но символы подчеркивания не являются объявлением "частных" данных: вы по-прежнему можете видеть эти имена и изменять их с помощью других форм импортирования, таких как инструкия `import`.

Альтернативный способ достижения эффекта сокрытия данных заключается в **присвоении на верхнем уровне модуля переменной `__all__` списка строк с именами переменных**.

Например,
```
# экспортируются только эти имена
__all__ = ['Error', 'encode', 'decode']
```

После этого инструкция `from *` будет копировать только имена, перечисленные в этом списке.

В действительности это соглашение, обратное соглашению `_X`

## Включение будущих возможностей языка

В языке периодически появляются изменения, которые могут повлиять на работоспособность существующего программного кода. Сначала они появляются в  виде расширений, которые по умолчанию отключены.

Чтобы включить такие расширения, используется инструкция импорта специального вида:
```
from __future__ import имя_функциональной_особенности
```

Эта инструкция вообще **должна появляться в самом начале файла модуля** (возможно, сразу же вслед за строкой документирования), потому что она **включает специальный режим компиляции программного кода для каждого отдельно взятого модуля**.

## Смешанные режимы использования: `__name__` и `__main__`

Ниже специальный прием, позволяющий импортировать файлы как модули и запускать их как самостоятельные программы.

Каждый модуль обладает **встроенным атрибутом `__name__`**, который устанавливается интерпретатором следующим образом:

* Если файл запускается как главный файл программы, атрибуту `__name__` на запуске присваивается значение `__main__`

* Если файл импортируется, атрибуту `__name__` присваивается имя модуля, под которым он будет известен клиенту.

Благодаря этому модуль может проверить собственный атрибут `__name__` и определить, был ли он запущен как самостоятельная программа или импортирован другим модулем. Например:

```
def tester():
    print("It's Christmas in Heaven...")
    
if __name__ == '__main__':  # Только когда запускается,
    tester()           # а не импортируется
```

Таким образом, переменная `__name__` может играть роль флага, определяющего режим использования.

Пожалуй, чаще всего проверка атрибута `__name__` выполняется в программном коде для **самопроверки модуля**. Вы можете добавить в конец модуля программный код, который будет выполнять проверку экспортируемых элементов внутри самого модулся, заключив этот код в условную инструкцию, проверяющую атрибут `__name__`.

При таком подходе вы можете использовать файл в других модулях, импортируя его, и тестировать логику работы, запуская его из командной строки или каким-либо другим способом.

### Тестирование модулей с помощью `__name__`

Скрипт, который находит минимальное значение среди множества предложенных аргументов:

```
def minmax(test, *args):
    res = args[0]
    for arg in args[1:]:
        if test(arg, res):
            res = arg
    return res
    
    def lessthan(x, y): return x < y
    def grtrthan(x, y): return x > y
    
    print(minmax(lessthan, 4, 2, 1, 5, 6, 3)) # Код самопроверки
    print(minmax(grtrthan, 4, 2, 1, 5, 6, 3))
```

При такой реализации результаты самопроверки будут выводиться на экран всякий раз, когда этот файл будет импортироваться для использования другим файлом.

Чтобы исправить, нужно обернуть проверочные вызовы функции в условную инструкцию, проверяющую `__name__`:

```
def minmax(test, *args):
    res = args[0]
    for arg in args[1:]:
        if test(arg, res):
            res = arg
    return res
    
def lessthan(x, y): return x < y
def grtrthan(x, y): return x > y

if __name__ = '__main__':
    print(minmax(lessthan, 4, 2, 1, 5, 6, 3)) # Код самопроверки
    print(minmax(grtrthan, 4, 2, 1, 5, 6, 3))
```

### Обработка аргументов командной строки с помощью `__name__`

Следующий модуль, `formats.py`, определяет вспомогательные функции форматирования строк. Кроме того, он проверяет имя модуля, чтобы узнать, был ли он запущен как самостоятельная программа. Если это так, он проверяет и **использует аргументы командной строки для запуска встроенного или конкретного теста**.

**Список `sys.argv` в языке Python содержит аргументы командной строки** – список строк со словами, введенными в  командной строке, где первый элемент списка всегда содержит имя файла сценария:

```
"""
Различные специализированные функции форматирования строк.
Модуль можно протестировать с помощью встроенных тестов или посредством передачи
аргументов командной строки.
"""

def commas(N):
    """
    Форматирует целое положительное число N, добавляя запятые,
    разделяющие группы разрядов: xxx,yyy,zzz
    """
    digits = str(N)
    assert(digits.isdigit())
    result = ''
    while digits:
        digits, last3 = digits[:-3], digits[-3:]
        result = (last3 + ',' + result) if result else last3
    return result

def money(N, width=0):
    """
    Форматирует число N, добавляя запятые, оставляя 2 десятичных знака
    в дробной части, добавляя в начало символ $ и знак числа, и,
    при необходимости, – отступ: $ -xxx,yyy.zz
    """
    sign = '-' if N < 0 else ''
    N = abs(N)
    whole = commas(int(N))
    fract = ('%.2f' % N)[-2:]
    format = '%s%s.%s' % (sign, whole, fract)
    return '$%*s' % (width, format)

if __name__ == '__main__':
    def selftest():
        tests = 0, 1 # ошибка при значениях: -1, 1.23
        tests += 12, 123, 1234, 12345, 123456, 1234567
        tests += 2 ** 32, 2 ** 100
        for test in tests:
            print(commas(test))

        print('')
        tests = 0, 1, -1, 1.23, 1., 1.2, 3.14159
        tests += 12.34, 12.344, 12.345, 12.346
        tests += 2 ** 32, (2 ** 32 + .2345)
        tests += 1.2345, 1.2, 0.2345
        tests += -1.2345, -1.2, -0.2345
        tests += -(2 ** 32), -(2**32 + .2345)
        tests += (2 ** 100), -(2 ** 100)
        for test in tests:
            print('%s [%s]' % (money(test, 17), test))

    import sys
    if len(sys.argv) == 1:
        selftest()
    else:
        print(money(float(sys.argv[1]), int(sys.argv[2])))
```

In [1]:
!python ./exercises/5_24/formats.py 

0
1
12
123
1,234
12,345
123,456
1,234,567
4,294,967,296
1,267,650,600,228,229,401,496,703,205,376

$             0.00 [0]
$             1.00 [1]
$            -1.00 [-1]
$             1.23 [1.23]
$             1.00 [1.0]
$             1.20 [1.2]
$             3.14 [3.14159]
$            12.34 [12.34]
$            12.34 [12.344]
$            12.35 [12.345]
$            12.35 [12.346]
$ 4,294,967,296.00 [4294967296]
$ 4,294,967,296.23 [4294967296.2345]
$             1.23 [1.2345]
$             1.20 [1.2]
$             0.23 [0.2345]
$            -1.23 [-1.2345]
$            -1.20 [-1.2]
$            -0.23 [-0.2345]
$-4,294,967,296.00 [-4294967296]
$-4,294,967,296.23 [-4294967296.2345]
$1,267,650,600,228,229,401,496,703,205,376.00 [1267650600228229401496703205376]
$-1,267,650,600,228,229,401,496,703,205,376.00 [-1267650600228229401496703205376]


In [3]:
!python ./exercises/5_24/formats.py 99999 0

$99,999.00


Когда этот модуль запускается как самостоятельная программа, он тестирует себя, как было описано выше, но при этом для управления тестированием он использует аргументы командной строки.

Можно использовать аргументы командной строки для обеспечения передачи в свои сценарии входных данных, которые также могут быть встроены в программный код как функции или классы и доступны импортирующим модулям.

Если требуются более широкие возможности обработки аргументов командной строки, стоит обратить внимание на модули стандартной библиотеки **`getopt` и `optparse`**

## Изменение пути поиска модулей

Сами программы на языке Python могут изменять путь поиска, изменяя встроенный список с именем `sys.path`. Этот список инициализируется во время запуска программы, однако и после этого допускается удалять, добавлять и изменять компоненты списка по своему усмотрению:

```
>>> import sys
>>> sys.path.append('/home')  # этот каталог теперь будет
                              # участвовать в поиске
```

Как только будут внесены изменения, они будут воздействовать на все последующие инструкции импорта, выполняемые в программе, так как все инструкции во всех файлах программы используют один и тот же общий список `sys.path`.

Таким образом, этот прием может использоваться для динамической настройки пути поиска внутри программ на языке Python.

Но если убрать из пути критически важный каталог, можно потерять доступ к критически важным утилитам.

>Такие изменения списка `sys.path` действуют только в рамках интерактивного сеанса или внутри программы (технически - в рамках процесса).

## Расширение `as` для инструкций `import` и `from`

Следующая инструкция `import`:
```
import longmodulename as name
```

эквивалентна инструкциям:

```
import longmodulename
name = longmodulename
del longmodulename  # не сохранять оригинальное имя
```

Это расширение обычно используется с целью создать короткие синонимы для длинных имен и избежать конфликтов с именами, уже используемыми в сценарии.

## Модули – это объекты: метапрограммы

Поскольку модули экспортируют большую часть своих свойств в виде встроенных атрибутов, это позволяет создавать **программы, которые управляют другими программами**. Такие менеджеры программ обычно называются **метапрограммами**.

Этот прием также называется **интроспекцией**, потому что программы могут просматривать внутреннее устройство объектов и действовать исходя из этого.

Следующие выражения представляют один и тот же атрибут и объект:

 * `M.name ` - полное имя объекта


 * `M.__dict__['name']` - доступ с использованием словаря пространства имен


 * `sys.modules['M'].name` - доступ через таблицу загруженных модулей


 * `getattr(M, 'name')` - доступ с помощью встроенной функции
 
Обеспечивая доступ к внутреннему устройству модулей, интерпретатор помогает создавать программы, управляющие другими программами.

## Импортирование модулей по имени в виде строки

Имя модуля в инструкции `import` и `from` является именем переменной. Тем не менее иногда ваша программа будет получать имя модуля, который следует импортировать, в виде строки во время выполнения.

Напрямую невозможно использовать строку как имя модуля, код `import "string"` выдаст ошибку, также если просто присвоить строку переменной.

Здесь интерпретатор попытается импортировать файл `x.py`, а не модуль `string`:

In [4]:
x = 'string'
import x

ModuleNotFoundError: No module named 'x'

Необходим специальный инструмент, выполняющий динамически загрузку модулей, имена которых создаются в виде строк во время выполнения.

Обычно используется **функция `exec` - компилирует строку в код**:

In [5]:
modname = 'string'
exec('import ' + modname)  # вып-ся как строка кода

Единственный настоящий недостаток `exec` в том, что она должна компилировать инструкцию `import` всякий раз, когда она запускается.

Можно использовать **встроенную функцию `__import__`, которая выполняет загрузку модуля, получая его имя в виде строки**. Результат получается тот же самый, но функция `__import__` **возвращает объект модуля**, поэтому его надо присвоить переменной, чтобы сохранить:

In [6]:
modname = 'string'
string = __import__(modname)

## Транзитивная перезагрузка модулей

Когда выполняется повторная загрузка модуля (с помощью `reload`), интерпретатор перезагружает только данный конкретный файл модуля - он не выполняет повторную загрузку модулей, которые были импортированы перезагружаемым модулем.

Например, если выполняется перезагрузка модуля A, и A импортирует модули B и C, перезагружен будет только модуль A. Инструкции внутри модуля A, которые импортируют B и C, будут перезапущены в процессе перезагрузки, но они просто вернут объекты уже загруженных модулей B и C.

Для перезагрузки всех модулей учше всего написать универсальный инструмент для транзитивной перезагрузки модулей, **сканируя содержимое атрибутов `__dict__` модулей и проверяя атрибут `type` в каждом элементе, чтобы отыскать вложенные модули**.

Такие вспомогательные функции могут **рекурсивно** вызывать сами себя и обходить произвольные цепочки импортирования.

Ниже пример модуля `reloadall.py`

```
"""
reloadall.py: транзитивная перезагрузка вложенных модулей
"""
import types
from imp import reload

def status(module):
    print('reloading ' + module.__name__)
    
def transitive_reload(module, visited):
    if not module in visited:  # Пропустить повторные посещения
        status(module)         # Перезагрузить модуль
        reload(module)         # И посетить дочерние модули
        visited[module] = None
        # Для всех атрибутов
        for attrobj in module.__dict__.values():
            if type(attrobj) == types.ModuleType:
                # Рекурсия, если модуль
                transitive_reload(attrobj, visited)

def reload_all(*args):
    visited = {}
    for arg in args:
        if type(arg) == types.ModuleType:
            transitive_reload(arg, visited)
            
if __name__ == '__main__':      # Тест: перезагрузить себя
    import reloadall            # перезагрузить этот модуль
    reload_all(reloadall)       # и модуль types
```

In [8]:
!python ./exercises/5_24/reloadall.py

reloading reloadall
reloading types
reloading functools
reloading collections.abc


## Концепции проектирования модулей

* **В языке Python вы всегда находитесь в модуле**. Нет никакого способа написать программный код, который не находился бы в каком-нибудь модуле. Фактически даже код, введенный в интерактивной оболочке, на самом деле относится к встроенному модулю с именем `__main__`.


* **Минимизируйте взаимозависимость модулей: глобальные переменные**. Как и функции, модули работают лучше, когда они написаны как самостоятельные закрытые компоненты. Следуйте правилу: модули должны быть максимально независимы от глобальных имен в других модулях.


* **Максимизируйте согласованность внутри модуля: общая цель**. Уменьшить взаимозависимость модулей можно за счет увеличения согласованности отдельного модуля - если все компоненты модуля используются для достижения общей цели, маловероятно, что такой модуль будет зависеть от внешних имен.


* **Модули должны редко изменять переменные в других модулях**. Использование глобальных переменных из других модулей - нормальное явление, но внесение изменений в глобальные переменные в других модулях служит признаком проблем с проектированием (конечно, есть и исключения), но нужно стремиться обмениваться данными через такие механизмы, как аргументы и возвращаемые значения функций.

## Типичные проблемы при работе с модулями

### Порядок следования инструкций на верхнем уровне имеет значение

* Инструкции программного кода на верхнем уровне в файле модуля (не вложенные в функцию) выполняются, как только интерпретатор достигает их в процессе импортирования. По этой причине он не может ссылаться на имена, присваивание которым производится позже


* Программный код внутри функций не выполняется, пока функция не будет вызвана, - разрешение имен внутри функций не производится до момента их вызова, поэтому они обычно могут ссылаться на имена, расположенные в любой части файла

Если вам необходимо объединять в  модуле программный код, выполняемый непосредственно, с инструкциями `def`, возьмите за правило помещать инструкции `def` в начало файла, а программный код верхнего уровня – в конец файла. При таком подходе ваши функции гарантированно будут определены к  моменту выполнения программного кода, который их использует.

### Инструкция `from` создает копии, а не ссылки

Инструкция `from` при выполнении присваивания именам в области видимости импортирующего модуля не создает синонимы, а копирует имена. Результат будет тем же самым, что и  для любых других инструкций присваивания в языке Python, но есть одно тонкое отличие, особенно когда программный код, использующий объекты совместно, находится в разных файлах.

Например, предположим, что у нас имеется следующий модуль (nested1.py):
```
# nested1.py
X = 99
def printer(): print(X)
```

**здесь `def printer()`, а не `def printer(X)`**!

Если импортировать эти два имени с помощью инструкции `from` в другом модуле (`nested2.py`), будут получены копии этих имен, а не ссылки на них. Изменение имени в импортирующем модуле приведет к изменениям только локальной версии этого имени, а имя в модуле `nested1.py` будет иметь прежнее значение:

```
# nested2.py
from nested1 import X, printer # Копировать имена
X = 88     # Изменит только локальную версию “X”!
printer()  # X в nested1 по-прежнему будет равно 99
```

In [10]:
!python ./exercises/5_24/nested2.py

99


Однако если выполнить импорт всего модуля с  помощью инструкции `import` и затем изменить значение с использованием полного имени, это приведет к изменению имени в файле `nested1.py`. Квалифицированное имя направляет интерпретатор к имени в указанном объекте модуля, а не к имени в импортирующем модуле `nested3.py`:
```
# nested3.py
import nested1 # Импортировать модуль целиком
nested1.X = 88 # OK: изменяется имя X в nested1
nested1.printer()
```

In [11]:
!python ./exercises/5_24/nested3.py

88


### Инструкция `from *` может затушевывать смысл переменных

Поскольку в инструкции `from module import *` не указываются необходимые имена переменных, она может непреднамеренно перезаписать имена, уже используемые в  области видимости импортирующего модуля.

Хуже того, это может осложнить поиск модуля, откуда исходит переменная, вызвавшая проблемы. Это особенно верно, когда форма инструкции `from *` используется более чем в одном импортированном файле.

### Функция `reload` может не оказывать влияния, если импорт осуществлялся инструкцией `from`

Инструкция `from` копирует (присваивает) имена при выполнении, поэтому нет никакой обратной связи с модулем, откуда были скопированы имена. Имена, скопированные инструкцией `from`, просто становятся ссылками на объекты, на которые ссылались по тем же именам в импортируемом модуле, когда была выполнена инструкция `from`.

Вследствие этого повторная загрузка импортируемого модуля может не оказывать воздействия на клиентов, которые импортировали его имена с помощью инструкции `from`. То есть имена в  модулях-клиентах будут по-прежнему ссылаться на оригинальные объекты, полученные инструкцией `from`, даже если имена в оригинальном модуле будут переопределены:

```
from module import X # X может не измениться в результате перезагрузки!
. . .
from imp import reload
reload(module)  # Изменится модуль, но не мои имена
X  # По-прежнему ссылается на старый объект
```

Чтобы сделать повторную загрузку более эффективной, используйте инструкцию `import` и  полные имена переменных вместо инструкции `from`. Поскольку полные имена всегда ведут обратно в импортированный модуль, после повторной загрузки они автоматически будут связаны с новыми именами в модуле.

```
import module # Получить объект модуля, а не имена
. . .
from imp import reload
reload(module) # Изменит непосредственно сам объект модуля
module.X # Текущее значение X: отражает результат перезагрузки
```

### `reload`, `from` и тестирование в интерактивной оболочке

Чтобы действительно получить доступ к новой версии функции, после перезагрузки модуля ее необходимо вызывать как `module.function` или повторно запустить инструкцию `from`:
```
from imp import reload
import module
reload(module)
from module import function
# Или оставить этот прием и использовать module.function()
function(1, 2, 3)
```

### Рекурсивный импорт с инструкцией `from` может не работать

В ходе импортирования инструкции в файле выполняются от начала и до конца, поэтому необходимо быть внимательнее, когда используются **модули, импортирующие друг друга** (эта ситуация называется **рекурсивным импортом**). Поскольку не все инструкции в модуле могут быть выполнены к моменту запуска процедуры импортирования другого модуля, некоторые из его имен могут оказаться еще не определенными.

Например:

```
# recur1.py
X = 1
import recur2  # Запустить recur2, если он еще не был имп-ан
Y = 2

# recur2.py
from recur1 import X  # OK: “X” уже имеет значение
from recur1 import Y  # Ошибка: “Y” еще не существует


C:\misc> C:\Python30\python
>>> import recur1
Traceback (innermost last):
  File “<stdin>”, line 1, in ?
  File “recur1.py”, line 2, in ?
    import recur2
  File “recur2.py”, line 2, in ?
    from recur1 import Y
ImportError: cannot import name Y
```

При рекурсивном импорте модуля `recur1` и модуля `recur2` интерпретатор не будет повторно выполнять инструкции модуля `recur1` (в противном случае это могло бы привести к бесконечному циклу), но пространство имен модуля `recur1` еще не заполнено до конца к моменту, когда он импортируется модулем `recur2`.

Решение? Не используйте инструкцию `from` в операции рекурсивного импорта (в самом деле!). Интерпретатор не зациклится, если вы все-таки сделаете это, но ваша программа попадет в зависимость от порядка следования инструкций в модулях.

**Два способа решения проблемы**:

* Можно ликвидировать рекурсивный импорт, правильно проектируя модули


* Если от циклов не удается избавиться полностью, попробуйте отсрочить обращение к именам модуля, используя инструкции `import` и полные имена (вместо инструкции `from`), или поместите инструкции `from` либо внуть функций (чтобы они не вызывались в программном коде верхнего уровня), либо ближе к концу файла, чтобы отложить их выполнение.